In [1]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("hotel_bookings_sample.csv")

# Calculate the average of the "adr" column
average_adr = df['adr'].mean()

# Print the result
print("Average ADR:", average_adr)


Average ADR: 131.95076


## langchain sql 

In [17]:
import pandas as pd
import sqlite3
from langchain.sql_database import SQLDatabase
# from langchain.utilities import SQLDatabaseChain
# from langchain.chat_models import ChatOpenAI

# Load CSV into Pandas
csv_file = "data\hotel_bookings_sample.csv"  # Replace with your CSV file path
df = pd.read_csv(csv_file)

# Create SQLite Database
db_path = "hotel_bookings_sample.db"
conn = sqlite3.connect(db_path)

# Store DataFrame as a Table
table_name = "my_table"  # Change as needed
df.to_sql(table_name, conn, if_exists="replace", index=False)

# Close the connection
conn.close()

print(f"CSV data successfully loaded into SQLite database: {db_path}")


CSV data successfully loaded into SQLite database: hotel_bookings_sample.db


In [18]:
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain_groq import ChatGroq

# Set up your Groq API key
os.environ["GROQ_API_KEY"] = "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"

# Connect to your SQLite database
db_path = "hotel_bookings_sample.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

# Create Groq LLM
# Options include: "llama3-8b-8192", "llama3-70b-8192", "mixtral-8x7b-32768", etc.
llm = ChatGroq(
    temperature=0,
    model_name="llama-3.3-70b-versatile"  # Choose the appropriate Groq model
)

# Create a SQL toolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Create the agent executor
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type="openai-tools",  # This still works with Groq models
)

# Run queries using natural language
response = agent_executor.invoke(
    {"input": "What is the average price of a hotel booking?"}
)
print(response["output"])

# # Ask more complex questions
# response = agent_executor.invoke(
#     {"input": "Find the top 5 values in column_x and their frequency."}
# )
# print(response["output"])



> Entering new SQL Agent Executor chain...
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded:   Then I should double check my query before running it to get the average price of a hotel booking.



my_table
Invoking: `sql_db_schema` with `{'table_names': 'hotel_bookings, hotels, users'}`
responded:   Then I should double check my query before running it to get the average price of a hotel booking.



Error: table_names {'hotel_bookings', 'hotels', 'users'} not found in database
Invoking: `sql_db_query_checker` with `{'query': 'SELECT AVG(price) FROM hotel_bookings LIMIT 10;'}`
responded:   Then I should double check my query before running it to get the average price of a hotel booking.



INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP

In [ ]:
import os
import sqlite3
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
#Replace with your actual database path
db_path = "hotel_bookings.db"

# Create sample database if it doesn't exist (for testing)
# if not os.path.exists(db_path):
#     conn = sqlite3.connect(db_path)
#     cursor = conn.cursor()
#     cursor.execute("CREATE TABLE sample (id INTEGER PRIMARY KEY, name TEXT, value INTEGER)")
#     cursor.execute("INSERT INTO sample VALUES (1, 'Item A', 100)")
#     cursor.execute("INSERT INTO sample VALUES (2, 'Item B', 200)")
#     cursor.execute("INSERT INTO sample VALUES (3, 'Item C', 300)")
#     conn.commit()
#     conn.close()
#     print(f"Created sample database at {db_path}")

# Connect to your SQLite database
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

# Create Ollama LLM connection - use your ngrok URL
OLLAMA_URL = "https://1c60-35-233-252-71.ngrok-free.app"  # Replace with your ngrok URL
llm = ChatOllama(
    model="qwen2.5-coder:14b",  # Use the SQLCoder model
    temperature=0,
    base_url=OLLAMA_URL
)

# Create a SQL toolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# Create the agent executor
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type="zero-shot-react-description",
    handle_parsing_errors=True,
)

# Run queries using natural language
response = agent_executor.invoke(
    {"input": "What is the impact of lead time on cancellation rates?"}
)
print("\nAgent response:")
print(response["output"])

## Llamaindex csv 


In [1]:
from llama_index.llms.groq import Groq
llm = Groq(model="llama-3.3-70b-versatile", api_key="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")
# response = llm.complete("Explain the importance of low latency LLMs")
# print(response)

c:\Users\shasw\miniconda3\envs\ENV3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
)
from llama_index.experimental.query_engine.pandas import (
    PandasInstructionParser,
)
# from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate

In [3]:
import pandas as pd

df = pd.read_csv("data\hotel_bookings.csv")

In [4]:
instruction_str = (
    "1. Convert the query to executable Python code using Pandas.\n"
    "2. The final line of code should be a Python expression that can be called with the `eval()` function.\n"
    "3. The code should represent a solution to the query.\n"
    "4. PRINT ONLY THE EXPRESSION.\n"
    "5. Do not quote the expression.\n"
)

pandas_prompt_str = (
    "You are working with a pandas dataframe in Python.\n"
    "The name of the dataframe is `df`.\n"
    "This is the result of `print(df.head())`:\n"
    "{df_str}\n\n"
    "Follow these instructions:\n"
    "{instruction_str}\n"
    "Query: {query_str}\n\n"
    "Expression:"
)
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n\n"
    "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
    "Pandas Output: {pandas_output}\n\n"
    "Response: "
)

pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str, df_str=df.head(5)
)
pandas_output_parser = PandasInstructionParser(df)
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

In [5]:
qp = QP(
    modules={
        "input": InputComponent(),
        "pandas_prompt": pandas_prompt,
        "llm1": llm,
        "pandas_output_parser": pandas_output_parser,
        "response_synthesis_prompt": response_synthesis_prompt,
        "llm2": llm,
    },
    verbose=True,
)
qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])
qp.add_links(
    [
        Link("input", "response_synthesis_prompt", dest_key="query_str"),
        Link(
            "llm1", "response_synthesis_prompt", dest_key="pandas_instructions"
        ),
        Link(
            "pandas_output_parser",
            "response_synthesis_prompt",
            dest_key="pandas_output",
        ),
    ]
)
# add link from response synthesis prompt to llm2
qp.add_link("response_synthesis_prompt", "llm2")

In [6]:
response = qp.run(
    query_str="What is the average price of a hotel booking?",
)

> Running module input with input: 
query_str: What is the average price of a hotel booking?

> Running module pandas_prompt with input: 
query_str: What is the average price of a hotel booking?

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
          hotel  is_canceled  lead_time  arrival_date_year arrival_date_...

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
> Running module pandas_output_parser with input: 
input: assistant: df['adr'].mean()

> Running module response_synthesis_prompt with input: 
query_str: What is the average price of a hotel booking?
pandas_instructions: assistant: df['adr'].mean()
pandas_output: 101.83112153446686

> Running module llm2 with input: 
messages: Given an input question, synthesize a respon

In [7]:
print(response.message.content)

The average price of a hotel booking is approximately $101.83.


## Llamaindex sql

In [1]:
from llama_index.llms.groq import Groq
llm = Groq(model="llama-3.3-70b-versatile", api_key="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")

c:\Users\shasw\miniconda3\envs\ENV3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
